In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
%cd /content/drive/My\ Drive/Colab\ stuff/sensor_anomaly

In [ ]:
def process_column(column):
    processed_column = column.copy()
    for index, state in enumerate(column[1:], 1):
        if state != state:
            processed_column[index] = processed_column[index-1]
    # print(processed_column)
    return processed_column
def process_activity_column(column):
    change_str = ""
    changing = False
    processed_column = column.copy()
    for index, state in enumerate(column):
        if str(state).split()[-1] == 'begin':
            changing = True
            if not change_str == '':
                change_str += '-'+ state.strip().split()[0]
            else:
                change_str = state.strip().split()[0]
            print(index, change_str)
        elif str(state).split()[-1] == 'end':
            end_act = state.strip().split()[0]
            acts = change_str.split('-')
            acts.remove(end_act)
            if not acts:
                print("no acts")
                changing = False
            change_str = '-'.join(acts)
            print(index, acts, change_str)
        if changing:
            processed_column[index] = change_str
    return processed_column

In [ ]:
# time_series_df.columns = ['ds', 'sensorID', 'state', 'activity']
# time_series_df['ds'] = time_series_df['ds'].dt.tz_localize(None)
# time_series_df['year'] = time_series_df.apply(lambda row: row.ds.year, axis=1)
# time_series_df['month'] = time_series_df.apply(lambda row: row.ds.month, axis=1)
# time_series_df['day'] = time_series_df.apply(lambda row: row.ds.day, axis=1)
# time_series_df['hours'] = time_series_df.apply(lambda row: row.ds.hour, axis=1)
# time_series_df['minutes'] = time_series_df.apply(lambda row: row.ds.minute, axis=1)

In [ ]:
data_milan = pd.read_csv("dataset/milan/data", sep = "\t", parse_dates=[0], infer_datetime_format=True, skip_blank_lines = True, names = ['Timestamp (UTC)', 'sensorID', 'state', 'activity'])
time_series_df = data_milan.sort_values(by="Timestamp (UTC)")
time_series_df = time_series_df.reset_index(drop=True)
df_sensors = pd.DataFrame(columns = time_series_df.sensorID.unique(),index = range(len(time_series_df)))
cols = list(time_series_df.sensorID.unique())
for index, row in time_series_df.iterrows():
    df_sensors.iat[index,cols.index(row.sensorID)] = row.state
for col in cols:
    df_sensors[col] = process_column(list(df_sensors[col]))
    df_sensors[col] = list(reversed(process_column(list(reversed(list(df_sensors[col]))))))

In [ ]:
df_sensors = df_sensors.replace({'ON':1, 'OFF':0, 'OPEN': 1, 'CLOSE': 0})
df_sensors['Activity'] = time_series_df['activity']
df_sensors['Activity'] = process_activity_column(list(df_sensors['Activity']))
df_sensors.Activity = df_sensors.Activity.fillna('None')
df_sensors.to_csv('dataset/milan/milan_processed.csv')
#test